In [1]:
from frame import Map, parse_init_frame
import numpy as np

In [ ]:
a = np.array([[1,2],[3,6],[7,5],[8,3]])
b = np.array([[2,5],[9,3],[4,2],[3,2],[7,4],[2,4]])
c = a[:, None] - b[ None,:]
c = np.linalg.norm(c,2,-1).min(0)
print(c)

In [2]:
input_string = """
1144 199346
9
1 43.75 49.25 0 12 1
2 45.75 49.25 0 0 1
3 47.75 49.25 0 0 1
4 43.75 47.25 -1 0 0
5 45.75 47.25 168 0 0
6 47.75 47.25 -1 0 0
7 44.75 45.25 -1 0 0
8 46.75 45.25 -1 0 0
9 46.25 42.25 -1 0 0
5 3 0.9657950401 1 0 0 0 -0.3755806088 47.5760498 47.40252686
-1 0 0 0 0 0 0 -0.006108176429 43.75140762 48.23157501
-1 0 0 0 0 0 0 0 3.25 2.25
-1 0 0 0 0 0 0 0 45.75 1.75
OK
"""
map_obj = parse_init_frame(input_string)

In [ ]:
def print_map(map_obj):# 遍历并打印Map对象的所有属性
    for w in map_obj.workbenches:
        for attr in dir(w):
            if not callable(getattr(w, attr)) and not attr.startswith("__"):
                print(f"workbenches对象属性 {attr}: {getattr(w, attr)}")

    # 遍历并打印Agent对象的所有属性
    for r in map_obj.robots:
        for attr in dir(r):
            if not callable(getattr(r, attr)) and not attr.startswith("__"):
                print(f"Agent对象属性 {attr}: {getattr(r, attr)}")

In [ ]:
map_obj.adj_mat

In [5]:
from task import TaskNode, dependency_dict, profit
import itertools
wb_id_to_type = [w.type_id for w in map_obj.workbenches]
eff_adj_mat = map_obj.workbench_adj_mat.copy()
for i, dists in enumerate(map_obj.workbench_adj_mat):
    i_wb_type = wb_id_to_type[i]
    for j,_ in enumerate(dists):
        j_wb_type = wb_id_to_type[j]
        deps = list(itertools.chain(*dependency_dict[i_wb_type]))
        if j_wb_type not in deps:
            eff_adj_mat[i,j] = 999
        else:
            eff_adj_mat[i,j] /= profit[i_wb_type]
task_node = TaskNode(map_obj.workbenches[8], wb_id_to_type, eff_adj_mat, map_obj)

In [6]:
sub_task = task_node.get_avail_sub_task()
for s in sub_task:
    src, tgt = s
    print(src.workbench.index, tgt.workbench.index)

1 5
2 5


In [8]:
task_node.children[0].workbench.type_id

6

In [ ]:
input_string = """
114451 189646
9
1 43.75 49.25 0 12 1
2 45.75 49.25 0 0 0
3 47.75 49.25 0 0 0
4 43.75 47.25 -1 0 0
5 45.75 47.25 168 0 0
6 47.75 47.25 -1 0 1
7 44.75 45.25 -1 0 0
8 46.75 45.25 0 23 1
9 46.25 42.25 -1 0 0
-1 0 0.9657950401 1 0 0 0 -0.3755806088 47.5760498 47.40252686
8 0 0 0 0 0 0 -0.006108176429 43.75140762 48.23157501
-1 0 0 0 0 0 0 0 3.25 2.25
-1 0 0 0 0 0 0 0 45.75 1.75
OK
"""
map_obj.update(input_string)
print_map(map_obj)

In [ ]:
from task import Scheduler
s = Scheduler(map_obj)

In [ ]:
s.src_to_tgt

In [ ]:
s.get_all_task()

In [ ]:
s.dispatch()

In [ ]:
map_obj.robots[2].task